# LLM

## Global

In [17]:
import torch
import random
import numpy as np

seed = 1337
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

## Dataset

### Load Dataset

In [18]:
with open("input.txt") as f:
    txt = f.read()
chars = sorted(list(set(txt)))
vocab_len = len(chars)
print("".join(chars))
print(vocab_len)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### Tokenisation / Detokenisation

In [19]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


### Dataloader

In [20]:
import torch
data = torch.tensor(encode(txt))
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [21]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target  = yb[b, t]
        print(f"Context: {context.tolist()}, Target: {target}")

Context: [53], Target: 59
Context: [53, 59], Target: 6
Context: [53, 59, 6], Target: 1
Context: [53, 59, 6, 1], Target: 58
Context: [53, 59, 6, 1, 58], Target: 56
Context: [53, 59, 6, 1, 58, 56], Target: 47
Context: [53, 59, 6, 1, 58, 56, 47], Target: 40
Context: [53, 59, 6, 1, 58, 56, 47, 40], Target: 59
Context: [49], Target: 43
Context: [49, 43], Target: 43
Context: [49, 43, 43], Target: 54
Context: [49, 43, 43, 54], Target: 1
Context: [49, 43, 43, 54, 1], Target: 47
Context: [49, 43, 43, 54, 1, 47], Target: 58
Context: [49, 43, 43, 54, 1, 47, 58], Target: 1
Context: [49, 43, 43, 54, 1, 47, 58, 1], Target: 58
Context: [13], Target: 52
Context: [13, 52], Target: 45
Context: [13, 52, 45], Target: 43
Context: [13, 52, 45, 43], Target: 50
Context: [13, 52, 45, 43, 50], Target: 53
Context: [13, 52, 45, 43, 50, 53], Target: 8
Context: [13, 52, 45, 43, 50, 53, 8], Target: 0
Context: [13, 52, 45, 43, 50, 53, 8, 0], Target: 26
Context: [1], Target: 39
Context: [1, 39], Target: 1
Context: [1,

In [24]:
xb.shape, yb.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

## Bigram Model

### Define the Model

In [49]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLM(nn.Module):
    def __init__(self, vocab_len):
        super().__init__()
        self.token_emb_table = nn.Embedding(vocab_len, vocab_len)
    
    def forward(self, idx, targets=None):
        logits = self.token_emb_table(idx)
        if not targets is None:
            B, T, C = logits.shape
            logits  = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
model = BigramLM(vocab_len)
out, loss = model(xb, yb)
print(out.shape, loss)

torch.Size([32, 65]) tensor(4.6435, grad_fn=<NllLossBackward0>)


In [65]:
def gen():
    print(
            decode(
            model.generate(
                torch.zeros(
                    (1, 1), dtype=torch.long
                ),
                max_new_tokens=100
            )[0].tolist()
        )
    )
gen()


3iOp'id:jewQU s$oUgZdZxZeMm:S$mwnhmdr
J&DL-:kpmPqAaLgfG UWoxsehGT'&lMlBx!$DtFB?:FXSke UzaO;t-Ug,Txlu


### Train

In [82]:
optim = torch.optim.AdamW(model.parameters(), lr=1e-3)
bs = 32
for epoch in range(1000):
    xb, yb = get_batch("train")
    logits, loss = model(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

print(loss.item())

2.4009902477264404


In [99]:
gen()



Toneavefl MH.
TEate e h be bayond hellakll baffl' t Oulemer he p idouirr wim; ckVashegowhe. ut, the
